In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp /content/drive/MyDrive/KAGGLE_API_CREDENTIALS/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d abtabm/multiclassimagedatasetairplanecar

In [ ]:
!unzip /content/multiclassimagedatasetairplanecar.zip

In [ ]:
import torch
from torch import nn
from sklearn.metrics import classification_report
from torchvision.transforms import ToTensor
from torchvision import datasets,transforms
import numpy as np
from torch.utils.data import DataLoader, SubsetRandomSampler

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomRotation(90),
    transforms.ToTensor()
])

test_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()

])

train_dataset = datasets.ImageFolder('Dataset/train', transform=train_transform)
test_dataset = datasets.ImageFolder('Dataset/test', transform=test_transform)

In [ ]:
#splitting datasets into batches

BATCH_SIZE = 32

train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE,shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE,shuffle=False)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
class Transport_Vehicle_Recognition(nn.Module):
  def __init__(self,input_shape,output_shape):
    super().__init__()
    self.layers_stack = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=input_shape, out_features=20),
        nn.BatchNorm1d(20),
        nn.ReLU(),
        nn.Linear(in_features=20, out_features=30),
        nn.BatchNorm1d(30),
        nn.ReLU(),
        nn.Linear(in_features=30, out_features=16),
        nn.BatchNorm1d(16),
        nn.ReLU(),
        nn.Linear(in_features=16, out_features=10),
        nn.BatchNorm1d(10),
        nn.ReLU(),
        nn.Linear(in_features=10,out_features=output_shape)
    )

  def forward(self,X):
    pred_probs = self.layers_stack(X)
    return pred_probs




In [ ]:
INPUT_SHAPE = 3 * 224 * 224
OUTPUT_SHAPE = len(train_dataset.classes)

model_0 = Transport_Vehicle_Recognition(input_shape=INPUT_SHAPE, output_shape=OUTPUT_SHAPE).to(device)

In [ ]:
#initializing loss function and optimizer

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model_0.parameters(), lr=0.08)

In [ ]:
import copy
import matplotlib.pyplot as plt

epochs= 150

#train and test loop
best_report = None
best_model_weights = model_0.state_dict()
best_acc = 0.0
train_losses  = []
test_losses  = []

for epoch in range(epochs):
  model_0.train()
  epoch_train_loss_sum = 0 #tracks the sum of train losses processed in a single epoch
  epoch_train_samples = 0 #tracks the number of samples seen in a single epoch

  for X_train,y_train in train_loader:
    X_train,y_train = X_train.to(device), y_train.to(device)
    y_train_pred = model_0(X_train)

    optimizer.zero_grad()
    train_loss = loss_fn(y_train_pred, y_train)

    epoch_train_loss_sum += train_loss.item() * X_train.size(0) # Accumulate total training loss for the epoch, considering batch sizes.
    epoch_train_samples += X_train.size(0) #adds the number of samples in the current batch to epoch_train_samples
    train_loss.backward()
    optimizer.step()


  epoch_avg_loss = epoch_train_loss_sum/epoch_train_samples
  train_losses.append(epoch_avg_loss)
  if epoch % 5 == 0:
    with torch.inference_mode():
        model_0.eval()
        epoch_trues = [] #stores all targeted labels in a single epoch
        epoch_preds = [] #stores all predicted labels in a single epoch
        epoch_test_loss = 0
        epoch_test_samples = 0
        for X_test, y_test in test_loader:
            X_test, y_test = X_test.to(device), y_test.to(device)
            y_test_pred = model_0(X_test)
            epoch_trues.extend(list(y_test.cpu().numpy()))
            epoch_preds.extend(list(y_test_pred.argmax(dim=1).cpu().numpy()))
            test_loss = loss_fn(y_test_pred, y_test)
            epoch_test_loss += test_loss.item() * X_test.size(0)
            epoch_test_samples += X_test.size(0)

        avg_test_loss = epoch_test_loss/epoch_test_samples
        test_losses.append(avg_test_loss)


        report = classification_report(y_true = epoch_trues,
                                       y_pred=epoch_preds,
                                       output_dict=True,
                                       target_names=test_dataset.classes, zero_division=0)

        acc = report['accuracy']
        if best_acc < acc:
          best_report = report
          best_model_weights = copy.deepcopy(model_0.state_dict())

  print(epoch)


plt.figure(figsize=(10,5))
plt.title("Train/Test Losses over Epochs")
plt.plot(train_losses, label='Train Loss')
plt.plot(test_losses, label='Test Loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(loc='upper right')
plt.show()
print(best_report)





In [ ]:
model_0.load_state_dict(best_model_weights)
torch.save(model_0.state_dict(),'Transport_Vehicle_Recognition')